# Выявления направлений развития нового продукта. SQL

# Содержание

- [Описание проекта. Общая информация о данных](#1)
- [Задача 1](#2)
- [Задача 2](#3)
- [Задача 3](#4)
- [Задача 4](#5)
- [Задача 5](#6)
- [Заключение](#7)

# Описание проекта. Общая информация о данных. <a id='1'></a>


## **Цель:**
**Проанализировать базу данных, для выявления направлений развития нового продукта**


## **Задачи:**

- Рассчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Рассчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


### Описание данных


In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 
# Чтобы выполнить SQL-запрос, используйте Pandas
# pd.io.sql.read_sql(query, con = engine)

Посмотрим на содержание таблиц

In [2]:
query = '''
SELECT *
FROM books
'''
books = pd.io.sql.read_sql(query, con = engine)
display(books.head())
print(books.info())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None


In [3]:
query = '''
SELECT *
FROM authors
'''
authors = pd.io.sql.read_sql(query, con = engine)
display(authors.head())

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''
SELECT *
FROM publishers
'''
publishers = pd.io.sql.read_sql(query, con = engine)
display(publishers.head())

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = '''
SELECT *
FROM ratings
'''
ratings = pd.io.sql.read_sql(query, con = engine)
display(ratings.head())

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''
SELECT *
FROM reviews
'''
reviews = pd.io.sql.read_sql(query, con = engine)
display(reviews.head())

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [7]:
print('Дубликатов в books:', books.duplicated().sum())
print('Дубликатов в authors:', authors.duplicated().sum())
print('Дубликатов в publishers:', publishers.duplicated().sum())
print('Дубликатов в ratings:', ratings.duplicated().sum())
print('Дубликатов в reviews:', reviews.duplicated().sum())

Дубликатов в books: 0
Дубликатов в authors: 0
Дубликатов в publishers: 0
Дубликатов в ratings: 0
Дубликатов в reviews: 0


# Задача 1 <a id='2'></a>

Рассчитать, сколько книг вышло после 1 января 2000 года

In [8]:
 #Определите издательство, которое выпустило наибольшее число книг толще 50 страниц
query = '''
SELECT COUNT(DISTINCT book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''
count_books = pd.io.sql.read_sql(query, con = engine)
display(count_books) 

,count
0,819


В базе данных содержится 819 книг вышедших после 1 января 2000 года.

# Задача 2 <a id='3'></a>

Для каждой книги посчитать количество обзоров и среднюю оценку

In [15]:
query = '''
WITH 
TABLE_CTE(book_id, rating_avg) AS
(
SELECT book_id, AVG(rating) AS rating_avg
FROM ratings
GROUP BY 1
),
TABLE_CTE1(book_id, review_count) AS
(
SELECT book_id, COUNT(review_id) AS review_count
FROM reviews
GROUP BY 1
)
SELECT title, t.rating_avg, t1.review_count
FROM books b
LEFT JOIN TABLE_CTE t ON b.book_id  = t.book_id
LEFT JOIN TABLE_CTE1 t1 ON b.book_id  = t1.book_id
ORDER BY 2 DESC
'''

rating_books = pd.io.sql.read_sql(query, con = engine)
display(rating_books) 


,title,rating_avg,review_count
0,A Dirty Job (Grim Reaper #1),5.00,4.0
1,Tai-Pan (Asian Saga #2),5.00,2.0
2,School's Out—Forever (Maximum Ride #2),5.00,3.0
3,Piercing the Darkness (Darkness #2),5.00,2.0
4,The War of Art: Break Through the Blocks & Win...,5.00,2.0
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,2.25,3.0
996,His Excellency: George Washington,2.00,2.0
997,Junky,2.00,2.0
998,Drowning Ruth,2.00,3.0


На 1000 книг из базы данных составлено от 1 до 7 обзоров, со средней оценкой 1,5 до 5.

# Задача 3 <a id='4'></a>

Определить издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [10]:
query = '''
SELECT publisher, COUNT(book_id)
FROM publishers p
JOIN books b ON p.publisher_id  = b.publisher_id
WHERE num_pages > 50
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1
'''

publisher = pd.io.sql.read_sql(query, con = engine)
display(publisher) 

,publisher,count
0,Penguin Books,42


По имеющейся базе данных наибольшее число книг (издания толще 50 страниц)-42 штуки, выпустило издательство Penguin Books.

# Задача 4 <a id='5'></a>

Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
query = '''
WITH 
TABLE_CTE(book_id, author_id, rating_count, rating_avg) AS
(
SELECT b.book_id AS book_id, b.author_id AS author_id, COUNT(rating) AS rating_count, AVG(rating) AS rating_avg
FROM ratings r
JOIN books b ON r.book_id  = b.book_id
GROUP BY 1
)
SELECT a.author, AVG(rating_avg)
FROM TABLE_CTE t
JOIN authors a ON t.author_id  = a.author_id
WHERE rating_count > 50
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1
'''

author = pd.io.sql.read_sql(query, con = engine)
display(author) 

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


J.K. Rowling/Mary GrandPré автор  с самой высокой средней оценкой книг — 4,28 (учитываются только книги с 50 и более оценками). 

# Задача 5 <a id='6'></a>

Рассчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
query = '''
WITH 
TABLE_CTE(username, rating_count) AS
(
SELECT username, COUNT(rating) AS rating_count
FROM ratings
GROUP BY 1
),
TABLE_CTE1(username, review_count) AS
(
SELECT r.username, COUNT(r.review_id) AS review_count
FROM TABLE_CTE t
JOIN reviews r ON r.username  = t.username
WHERE t.rating_count > 50
GROUP BY 1
)
SELECT AVG(review_count)
FROM TABLE_CTE1
'''

reviews_avg = pd.io.sql.read_sql(query, con = engine)
display(reviews_avg) 

,avg
0,24.333333


Наиболее активные пользователи публикуют в среднем 24 обзора.

# Заключение <a id='7'></a>

- Дубликатов в таблицах нет.
- В базе данных содержится 819 книг вышедших после 1 января 2000 года.
- На 1000 книг из базы данных составлено от 1 до 7 обзоров, со средней оценкой 1,5 до 5.
- По имеющейся базе данных наибольшее число книг (издания толще 50 страниц)-42 штуки, выпустило издательство Penguin Books.
- J.K. Rowling/Mary GrandPré автор  с самой высокой средней оценкой книг — 4,28 (учитываются только книги с 50 и более оценками).
- Наиболее активные пользователи публикуют в среднем 24 обзора.